In [24]:
import sys
sys.path.append(r'D:\PyCharmProjects\VFPUMC02')
sys.path.append(r'C:\Users\Administrator\PycharmProjects\VFPUMC02')
sys.path.append(r'/root/VFPUMC02')
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
from datasets.DataSet import BankDataset
from utils.DataProcessUtils import *
from consts.Constants import DATASETS_PATH
import os
from datasets.VFPU_GEN_DATASET import VFPU_GEN_DATASET
from enums.DataSetsName import DataSetsName
from ctgan import CTGAN
from classfiers.TwoStep import TwoStep
import pandas as pd
from vf4lr.VF_LR import VF_LR
from classfiers.VF_LinearRegression import VF_LinearRegression
from classfiers.VF_TwoStep import VF_TwoStep
from utils.FateUtils import convert_ipynb_to_py
from gen.VFPU_GEN import VFPU_GEN
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge

In [22]:
dataset = VFPU_GEN_DATASET()
df_A, df_B, y = dataset.get_dataset_parts(DataSetsName.BANK.value)
miss_rate=0.2
complete_df_B, incomplete_df_B, missing_df_B = constru_row_miss_df(df_B, miss_rate)

In [25]:
df_A.shape

(41188, 31)

In [23]:
incomplete_df_B.shape

(32951, 32)

In [9]:
ctgan = CTGAN(epochs=5)

clf =  LogisticRegression(solver='sag', max_iter=5)
reg =  Ridge(alpha=0.1, solver='sag', max_iter=5)
two_step = TwoStep(
    base_classifier=clf,
    base_regressor=reg,
    max_iter=5,
    k=0.25
)

vf_clf = VF_LR(learning_rate=0.2,epoch_num=5,batch_size=64)
vf_reg = VF_LinearRegression(config={'lr': 0.01,'lambda': 0.1,'n_iters':5})
vf_two_step = VF_TwoStep(
    clf=vf_clf,
    reg=vf_reg,
    k=0.25,
    max_iter=5
)

2025-01-23 23:59:00,741 - ConsoleLogger - INFO - TwoStep 类已初始化: max_iter=5, k=0.25, base_classifier=LogisticRegression, base_regressor=Ridge
2025-01-23 23:59:00,741 - INFO - TwoStep 类已初始化: max_iter=5, k=0.25, base_classifier=LogisticRegression, base_regressor=Ridge
2025-01-23 23:59:00,742 - ConsoleLogger - INFO - VF_LR 模型初始化完成。
2025-01-23 23:59:00,742 - INFO - VF_LR 模型初始化完成。
2025-01-23 23:59:00,743 - ConsoleLogger - INFO - 模型配置: {'learning_rate': 0.2, 'epoch_num': 5, 'batch_size': 64, 'client_num': 2}
2025-01-23 23:59:00,743 - INFO - 模型配置: {'learning_rate': 0.2, 'epoch_num': 5, 'batch_size': 64, 'client_num': 2}
2025-01-23 23:59:00,744 - ConsoleLogger - INFO - VF_LinearRegression 模型初始化完成。
2025-01-23 23:59:00,744 - INFO - VF_LinearRegression 模型初始化完成。
2025-01-23 23:59:00,745 - ConsoleLogger - INFO - 模型配置: {'lr': 0.01, 'lambda': 0.1, 'n_iters': 5}
2025-01-23 23:59:00,745 - INFO - 模型配置: {'lr': 0.01, 'lambda': 0.1, 'n_iters': 5}
2025-01-23 23:59:00,746 - INFO - VF_TwoStep 类已成功初始化: max_iter=

In [10]:
vfpu_gen = VFPU_GEN(two_step,vf_two_step,ctgan,df_A,incomplete_df_B)

/root/miniconda3/envs/fate_env/lib/python3.10/site-packages/torch/autograd/graph.py:825: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at ../aten/src/ATen/cuda/CublasHandlePool.cpp:135.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


In [18]:
m = incomplete_df_B.shape[1]
# 生成 k 列， 预测 m - k 列
for k in range(m + 1):
    predict_cols = incomplete_df_B.columns[:m-k]
    vfpu_gen.fit(predict_cols)
    rmse, mse, mae, r2 = evaluate_imputed_data(missing_df_B, synthetic_data)